In [1]:
import time
import numpy as np
import pandas as pd

In [2]:
n=10   # number of inputs

beta=np.zeros(n) ## first and last parameters are 1, others are zeros
beta[1]=1   
beta[-1]=1
print(beta)
beta.shape=(n,1)
print(beta)
print("-----")
x=np.random.rand(1,n)[0]
print("x: ", x)
x_T_beta = x.dot(beta)
w = 0.1*np.random.rand(1)[0]
print("x_t_beta : ", x_T_beta)
print("w : ", w)
#y=float(x.dot(beta))+0.1*np.random.rand(1)[0] ## y =x^T beta +w
y=float(x_T_beta)+w ## y =x^T beta +w
print("y : ", y)
message=np.array2string(np.append([0,y],x),separator=",") 
print(message)

[0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
[[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]]
-----
x:  [0.5850245  0.57933072 0.67363201 0.80047994 0.25907805 0.9099468
 0.09312666 0.81951488 0.49311056 0.37880294]
x_t_beta :  [0.95813366]
w :  0.010612732834882455
y :  0.9687463959230191
[0.        ,0.9687464 ,0.5850245 ,0.57933072,0.67363201,0.80047994,
 0.25907805,0.9099468 ,0.09312666,0.81951488,0.49311056,0.37880294]


In [3]:
def prepareDataFrame(data_file):
    #Takes about one minute to load
    data=pd.read_csv(data_conv,header=None,sep=" ")
    data.columns=["Date","Hour","Sensor","Value","Voltage"]
    data=data.sort_values(['Date','Hour']).reset_index(drop=True)
    
    data['datetime']=pd.to_datetime(data.Date+' '+data.Hour)
    data['relative_datetime']=data['datetime']-data['datetime'][0]
    data['seconds']=data['relative_datetime'].dt.total_seconds()
    
    sensorId_type=data.Sensor.str.split("-",expand=True)
    sensorId_type.columns=['SensorId','Type']
    data['SensorId']=sensorId_type['SensorId'].astype(int)
    data['Type']=sensorId_type['Type'].astype(int)
    
    #Drop features not needed for the simulation
    data=data.drop(['datetime','relative_datetime','Sensor','Date','Hour','Voltage'],axis=1)
    return data

In [4]:
data_conv = "../data/data.conv.txt"
data = prepareDataFrame(data_conv)

In [5]:
temp_1=data[(data.SensorId==1) & (data.Type==0) & (data.seconds<=8*86400)]
temp_1=temp_1.reset_index(drop=True)

In [6]:
temp_24=data[(data.SensorId==24) & (data.Type==0) & (data.seconds<=8*86400)]
temp_24=temp_24.reset_index(drop=True)

In [7]:
# 5 closest neighbors of sensor 1 are sensors 2, 3, 33, 34, 35
neighbors_1 = [data[(data.SensorId==2) & (data.Type==0) & (data.seconds<=8*86400)],\
               data[(data.SensorId==3) & (data.Type==0) & (data.seconds<=8*86400)],\
               data[(data.SensorId==33) & (data.Type==0) & (data.seconds<=8*86400)],\
               data[(data.SensorId==34) & (data.Type==0) & (data.seconds<=8*86400)],\
               data[(data.SensorId==35) & (data.Type==0) & (data.seconds<=8*86400)]]

for neighbor in neighbors_1:
    

In [8]:
# 5 closest neighbors of sensor 24 are sensors 22, 23, 25, 26, 27
neighbors_24 = [data[(data.SensorId==22) & (data.Type==0) & (data.seconds<=8*86400)],\
                data[(data.SensorId==23) & (data.Type==0) & (data.seconds<=8*86400)],\
                data[(data.SensorId==25) & (data.Type==0) & (data.seconds<=8*86400)],\
                data[(data.SensorId==26) & (data.Type==0) & (data.seconds<=8*86400)],\
                data[(data.SensorId==27) & (data.Type==0) & (data.seconds<=8*86400)]]

In [13]:
def convertTimeToSlots(dataframe):
    interval_slot = 30
    #divide data in slots of 30sec, add each slot value to each entry
    dataframe["slot"] = (dataframe["seconds"]//interval_slot).astype(int)
    #transform the seconds so that for each slot, its corresponding 'seconds' value is at the center of this slot (usefull for plots)
    dataframe["seconds"] = interval_slot*(dataframe["slot"] + dataframe["slot"]+1) / 2
    
    #Take care of the potential multiple value appearing within the same slot -> average them
    dataframe = dataframe.groupby(["slot"]).agg("mean")
    dataframe.reset_index(level=0, inplace=True)
    return dataframe

def fillMissingRows(dataframe):
    interval_slot = 30
    nb_slots = 23040
    missing_values = {"slot": [], "seconds": [], "SensorId": [], "Type": []}
    for i in range (nb_slots): #total nb of slots for 8 days
        if i not in dataframe["slot"].values:
            seconds = interval_slot*(i + i+1) / 2
            missing_values["slot"].append(i)
            missing_values["seconds"].append(seconds)
            missing_values["SensorId"].append(1)
            missing_values["Type"].append(0)
    #Build DataFrame with missing values
    temp_missing = pd.DataFrame(missing_values)
    #Merge the two Dataframe and sort them by values of the 'slot' column
    #At this point, the temperature values are still missing -> NaN
    complete_temp = dataframe.append(temp_missing).sort_values('slot')
    #Replace NaN by values extracted from a linear method based on the neighbors
    complete_temp["Value"] = complete_temp["Value"].interpolate(limit_direction="both")    
    return complete_temp

def preprocessDataFrames(output_df, input_dfs):
    output_df = fillMissingRows(convertTimeToSlots(output_df))
    for i in range (len(input_dfs)):
        input_dfs[i] = fillMissingRows(convertTimeToSlots(input_dfs[i]))
    merged_inputs_dfs = pd.concat(input_dfs) 
    return output_df, merged_inputs_dfs


In [14]:
temp_1, merged_neighbors_1 = preprocessDataFrames(temp_1, neighbors_1)    



/home/kais/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [15]:
output = temp_1[temp_1.slot == 0]
print(output)
input_df= merged_neighbors_1[merged_neighbors_1.slot == 0]
print(input_df)

   SensorId  Type    Value  seconds  slot
0       1.0   0.0  19.2436     15.0   0.0
   SensorId  Type    Value  seconds  slot
0       2.0   0.0  19.6160     15.0   0.0
0       3.0   0.0  19.5278     15.0   0.0
0      33.0   0.0  19.1456     15.0   0.0
0       1.0   0.0  18.4988     15.0   0.0
0      35.0   0.0  19.4200     15.0   0.0


In [16]:
def applyRLS(output_df, inputs_dfs):
    nb_slots = 23040
    nb_features = len(inputs_dfs)
    betas = np.zeros(nb_features)
    covar_matrix = np.diag(np.zeros(n)+1)
    forgetting_factor = 1.0
    predictions = []
    for slot in range (nb_slots):
        output = output_df[output_df.slot == slot].Value
        inputs = [input_df[input_df.slot == slot].Value for input_df in inputs_dfs]

            beta=state[1]
        beta.shape=(n,1)
        V=state[2]
        mu=state[3]
        sse=state[4]  ## sum of squared errors
        N=state[5]   ## number of treated samples
        x.shape=(1,n)
        err=y-x.dot(beta)
        sse=sse+pow(err,2.0)
        V=1.0/mu*(V-V.dot(x.T).dot(x).dot(V)/(1.0+float(x.dot(V).dot(x.T))))
        gamma=V.dot(x.T)
        beta=beta+gamma*err
        
        return (key,beta,V,mu,sse/(N+1.0),N+1)  ## update formula mod1
    
#def mergeByslots(temp_data):
#def mapSecondsToSlots(temperature_data, neighbors_data):
#    interval_slot = 30  #30s slots
#    nb_slots = int(86400/interval_slot)
#    print(nb_slots)
#    for day in range (0,8):
        

#def predictSlots(temperature_data, neighbors_data):
#    interval_slot = 30  #30s slots
#    nb_slots = int(86400/interval_slot)
#    print(nb_slots)
    
    


In [12]:
print(np.diag(np.zeros(n)+10) )

[[10.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 10.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 10.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 10.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 10.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 10.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 10.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 10.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 10.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 10.]]
